In [1]:
import time

from IPython.display import Markdown, display  # Import
from openai import OpenAI
import warnings
import tiktoken
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
from dotenv import load_dotenv
import asyncio
import pandas as pd
from io import StringIO
from src.business_requirements import *
from src.prompts import *
from src.simplified_business_requirements import *
from src.detailed_business_requirements import *
warnings.filterwarnings("ignore")
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")


## Query Question

In [2]:
query = "What are the business requirements for Regulation O?"
response = verbatim_business_requirements(query, top_k=25)
display(Markdown(response))

Business Requirement^^
"No member bank may extend credit to any insider of the bank or insider of its affiliates unless the extension of credit is made on substantially the same terms as those prevailing at the time for comparable transactions by the bank with other persons not covered by this part and does not involve more than the normal risk of repayment or present other unfavorable features."^^
"No member bank may extend credit to any insider of the bank or insider of its affiliates in an amount that, when aggregated with the amount of all other extensions of credit to that person and to all related interests of that person, exceeds the higher of $25,000 or five percent of the member bank's unimpaired capital and unimpaired surplus, unless the extension of credit has been approved in advance by a majority of the entire board of directors and the interested party has abstained from participating directly or indirectly in the voting."^^
"In no event may a member bank extend credit to any insider of the bank or insider of its affiliates in an amount that, when aggregated with all other extensions of credit to that person and to all related interests of that person, exceeds $500,000, except by complying with the requirements of this part."^^
"No executive officer, director, or principal shareholder shall knowingly receive, or knowingly permit any of that person's related interests to receive, from a member bank, directly or indirectly, any extension of credit not authorized under this part."^^
"Each executive officer or director of a member bank the shares of which are not publicly traded shall report annually to the board of directors of the member bank the outstanding amount of any credit that was extended to the executive officer or director and that is secured by shares of the member bank."^^
"No member bank may extend credit to any of its executive officers, and no executive officer of a member bank shall borrow from or otherwise become indebted to the bank, except in the amounts, for the purposes, and upon the conditions specified in paragraphs (c) and (d) of this section."^^
"A member bank is authorized to extend credit to any executive officer of the bank for (1) financing the education of the executive officer's children; (2) financing or refinancing the purchase, construction, maintenance, or improvement of a residence of the executive officer, provided that the extension is secured by a first lien on the residence and the residence is owned (or expected to be owned) by the executive officer; or (3) for any other purpose not specified above, provided that the aggregate amount of such extensions does not exceed at any one time the higher of 2.5 percent of the bank's unimpaired capital and unimpaired surplus or $25,000, but in no event more than $100,000."^^
"Any extension of credit by a member bank to any of its executive officers shall be promptly reported to the member bank's board of directors and made subject to the condition in writing that the extension of credit will, at the option of the member bank, become due and payable at any time that the officer is indebted to any other bank or banks in an aggregate amount greater than the amount specified for a category of credit."^^
"Any member bank, or any officer, director, employee, agent, or other person participating in the conduct of the affairs of the bank that violates any provision of this part (other than Sec. 215.9) is subject to civil penalties as specified in section 29 of the Federal Reserve Act."^^
"The requirements of this part apply to savings associations in the same manner and to the same extent as if the savings association were a member bank."

## Create Excel Sheet

In [3]:
def clean_dataframe(df, column):
    """
    Cleans the DataFrame by:
      - Dropping rows in which the specified column does not contain any alphabetical characters (A-Z, a-z).
      - Removing the substring "| " from the specified column.

    Parameters:
      df (pd.DataFrame): The input DataFrame.
      column (str): The name of the column to clean.

    Returns:
      pd.DataFrame: The cleaned DataFrame.
    """
    # Ensure the column values are treated as strings.
    df[column] = df[column].astype(str)

    # Keep only rows where the column contains at least one alphabetical character.
    df = df[df[column].str.contains(r'[A-Za-z]', na=False)]

    # Remove the exact substring "| " from the column.
    df[column] = df[column].str.replace("| ", "", regex=False)

    # Remove the exact substring "|" from the column.
    df[column] = df[column].str.replace("|", "", regex=False)

    # Remove the exact substring "^^" from the column.
    df[column] = df[column].str.replace("^^", "", regex=False)

    # Remove the exact substring "" " from the column.
    df[column] = df[column].str.replace('''"''', "", regex=False)
    return df

df = convert_str_to_df(response)

# Clean Table
df = df.dropna(axis=1, how='all')
df = clean_dataframe(df, df.columns[0])
df["Business_Requirements"] = df[df.columns[0]]
df = df.drop(df.columns[0], axis=1)
df


,Business_Requirements
0,No member bank may extend credit to any inside...
1,No member bank may extend credit to any inside...
2,In no event may a member bank extend credit to...
3,"No executive officer, director, or principal s..."
4,Each executive officer or director of a member...
5,No member bank may extend credit to any of its...
6,A member bank is authorized to extend credit t...
7,Any extension of credit by a member bank to an...
8,"Any member bank, or any officer, director, emp..."
9,The requirements of this part apply to savings...


## Create Simplified Business Requirements

In [4]:
async def simplified_business_requirement(query: str, df: pd.DataFrame):
    # Create asynchronous tasks for each row's business requirement
    tasks = [
        simplified_buss_req(original_question=query, business_requirement=row)
        for row in df["Business_Requirements"]
    ]

    # Execute all tasks concurrently and collect results
    results = await asyncio.gather(*tasks)

    # Store the simplified business requirements back into the DataFrame
    df["Simplified_Business_Requirements"] = results
    return df

# Run the asynchronous function with your query and DataFrame
await simplified_business_requirement(query=query, df=df)
df

,Business_Requirements,Simplified_Business_Requirements
0,No member bank may extend credit to any inside...,The business requirements for Regulation O sta...
1,No member bank may extend credit to any inside...,The business requirements for Regulation O sta...
2,In no event may a member bank extend credit to...,The business requirement for Regulation O stat...
3,"No executive officer, director, or principal s...",The business requirement for Regulation O stat...
4,Each executive officer or director of a member...,The business requirement for Regulation O stat...
5,No member bank may extend credit to any of its...,The business requirements for Regulation O sta...
6,A member bank is authorized to extend credit t...,The business requirements for Regulation O sta...
7,Any extension of credit by a member bank to an...,The business requirements for Regulation O sta...
8,"Any member bank, or any officer, director, emp...",The business requirements for Regulation O sta...
9,The requirements of this part apply to savings...,The business requirements for Regulation O sta...


## Obtain Detail Requirements

In [5]:
async def detailed_business_requirement(query: str, df: pd.DataFrame):
    # Create asynchronous tasks for each row's business requirement
    tasks = [
        detailed_buss_req(original_question=query, business_requirement=row)
        for row in df["Business_Requirements"]
    ]

    # Execute all tasks concurrently and collect results
    results = await asyncio.gather(*tasks)

    # Store the simplified business requirements back into the DataFrame
    df["Detailed_Business_Requirements"] = results
    return df

# Run the asynchronous function with your query and DataFrame
await detailed_business_requirement(query=query, df=df)
df

,Business_Requirements,Simplified_Business_Requirements,Detailed_Business_Requirements
0,No member bank may extend credit to any inside...,The business requirements for Regulation O sta...,- **Identify Insiders**: Clearly define who qu...
1,No member bank may extend credit to any inside...,The business requirements for Regulation O sta...,- **Identify Insiders**: Clearly define who qu...
2,In no event may a member bank extend credit to...,The business requirement for Regulation O stat...,- **Identify Insiders**: Clearly define who qu...
3,"No executive officer, director, or principal s...",The business requirement for Regulation O stat...,- **Identify Key Individuals and Entities**\n ...
4,Each executive officer or director of a member...,The business requirement for Regulation O stat...,- **Identify Executive Officers and Directors*...
5,No member bank may extend credit to any of its...,The business requirements for Regulation O sta...,To address the Verbatim Business Requirements ...
6,A member bank is authorized to extend credit t...,The business requirements for Regulation O sta...,- **Identify Eligible Credit Purposes:**\n - ...
7,Any extension of credit by a member bank to an...,The business requirements for Regulation O sta...,- **Identify Executive Officers**: Clearly def...
8,"Any member bank, or any officer, director, emp...",The business requirements for Regulation O sta...,To address the Verbatim Business Requirements ...
9,The requirements of this part apply to savings...,The business requirements for Regulation O sta...,- **Identify Applicable Entities**\n - Determ...


##

In [6]:
df.to_excel("../Data/Results/Query_Results.xlsx")